In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime,timedelta
import stock_info as si
import joblib


In [82]:
class StockPricePredictor(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size = 100, num_layers=2, output_size=1):
        super(StockPricePredictor, self).__init__()

        #LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)

        #Dropout layer
        self.dropout = nn.Dropout(0.2)

        #fully Connected Layer
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        out,_ = self.lstm(x)
        out = self.dropout(out)
        out = self.fc(out[:,-1,:])
        return out

In [83]:
# Load the model
model = StockPricePredictor(input_size=3, hidden_layer_size=100, num_layers=2, output_size=1)
model.load_state_dict(torch.load('stock_price_predictor.pth'))
model.eval()  # Set model to evaluation mode

# Load the scaler
scaler = joblib.load('scaler.pkl')

print("Model and Scaler loaded successfully.")


Model and Scaler loaded successfully.


C:\Users\DivyaPrakash\AppData\Local\Temp\ipykernel_28364\940267892.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('stock_price_predicto

In [84]:
#plot historical data.
end_date = datetime.today()
start_date = end_date - timedelta(days=150) 
ticker = "LEMONTREE.NS"

data = si.get_historical_data(start_date=start_date, end_date=end_date, ticker=ticker)
data = data[[f'Close_{ticker}']]

# #calculate moving avg 
data['MAS'] = data[f'Close_{ticker}'].rolling(window=5).mean()
data['MAS20'] = data[f'Close_{ticker}'].rolling(window=20).mean()
# data.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed

data shape check:(100, 5)


In [85]:
df_sorted = data.sort_index(ascending=False)
df_sorted = df_sorted.head(60)

In [86]:
df_sorted

,Close_LEMONTREE.NS,MAS,MAS20
Date,,,
2025-04-11,139.750000,137.162000,134.895001
2025-04-09,136.800003,137.950000,134.210001
2025-04-08,138.639999,138.732001,133.723501
2025-04-07,133.789993,138.416000,133.242001
2025-04-04,136.830002,137.340002,133.079001
2025-04-03,143.690002,136.090002,132.657501
2025-04-02,140.710007,133.724002,131.710001
2025-04-01,137.059998,132.874002,130.967500
2025-03-28,128.410004,133.236002,130.200000


In [87]:

# Scale the new data using the loaded scaler
scaled_new_data = scaler.transform(df_sorted)

# Convert the scaled data to a tensor
scaled_new_data_tensor = torch.tensor(scaled_new_data.reshape(1, 60, 3), dtype=torch.float32)

# Predict using the loaded model
model.eval()
with torch.no_grad():
    predicted_price = model(scaled_new_data_tensor).numpy()
print(predicted_price)
# Inverse scale the prediction
predicted_price = scaler.inverse_transform(np.concatenate([predicted_price, np.zeros((predicted_price.shape[0], 2))], axis=1))

# Extract the predicted Close price
predicted_close = predicted_price[0][0]

print(f"Predicted Close Price: {predicted_close}")


[[0.896035]]
Predicted Close Price: 143.9845877422341


In [88]:
scaled_new_data_tensor.shape

torch.Size([1, 60, 3])